In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

gauss2 = np.loadtxt('../Data/2gaussian.txt')
gauss3 = np.loadtxt('../Data/3gaussian.txt')

In [55]:
prob = lambda x, mu, sigma, w: w*multivariate_normal.pdf(x, mu, sigma)
def e(x, params, k):
    members = [[] for i in range(x.shape[0])]
    for i in range(x.shape[0]):
        for j in range(k):
            members[i].append(prob(x[i], params['mu'][j], params['sigma'][j], params['w'][j]))
        members[i] = members[i]/sum(members[i])
#     print('mem', members)
#     members = np.array(members)
    ll = sum(np.log(np.sum(members, axis = 1)))
    return np.array(members), ll

def m(x, params, members, k):
    ns = np.sum(members, axis=0)
#     print('members ', members)
#     print('nk ', ns)
    params['w'] = np.divide(ns, len(x))
    params['mu'] = [
        np.divide(np.sum(np.multiply(members.T[i].reshape(-1, 1), x), axis=0), ns[i]) for i in range(k)]
    params['sigma'] = [
        np.divide(np.dot((x - params['mu'][i]).T,np.multiply(members.T[i].reshape(-1, 1),(x - params['mu'][i]))), np.sum(members, axis=0)[i]) for i in range(k)
    ]
    return params

def em(x, params, k, threshold=1e-8):
    iter = 0
    ll_lst = list()
    while True:
        iter+=1
        mem, ll = e(x, params, k)
        params = m(x, params, mem, k)
#         print(iter, ll, params)
        if iter > 2 and np.isclose(ll_lst[-1], ll, atol=threshold):
            break
        ll_lst.append(ll)
    print('#iters: {}'.format(iter))
    print('params: \n{}'.format(params))

### Two Gaussian

In [57]:
initial = {
    'w': [0.4, 0.6],
    'mu': [[2.8, 2.9], [6.8, 3.8]],
    'sigma': [[[1.000, 0.023], [0.023, 2.877]],
               [[1.000, 0.483], [0.423, 0.997]]]}
em(gauss2, initial, 2)

#iters: 3
params: 
{'w': array([ 0.33115045,  0.66884955]), 'mu': [array([ 2.97295552,  3.05438516]), array([ 7.00172864,  3.97692683])], 'sigma': [array([[ 0.9761496 ,  0.03037211],
       [ 0.03037211,  2.95470502]]), array([[ 0.99523386,  0.50847154],
       [ 0.50847154,  1.01013246]])]}


### Three Gaussian

In [58]:
initial = {
    'w': [0.3, 0.3, 0.4],
    'mu': [[3.00, 3.00], [7.00, 4.00], [5.0, 7.0]],
    'sigma': [[[1.000, 0.023], [0.023, 3.387]],
              [[0.900, 0.500], [0.500, 0.900]],
              [[0.900, 0.180], [0.180, 0.900]],]}
em(gauss3, initial, 3)

#iters: 3
params: 
{'w': array([ 0.21704004,  0.29702647,  0.48593349]), 'mu': [array([ 3.08685134,  3.1852099 ]), array([ 7.03245271,  4.02116763]), array([ 5.03627814,  7.02134354])], 'sigma': [array([[ 1.06291983,  0.1129919 ],
       [ 0.1129919 ,  3.6613992 ]]), array([[ 0.97190725,  0.49385651],
       [ 0.49385651,  0.99423992]]), array([[ 0.94932301,  0.16781069],
       [ 0.16781069,  0.94784999]])]}
